## 层次分析法

层次分析法原理及计算过程详解 - 子木的文章 - 知乎  https://zhuanlan.zhihu.com/p/266405027

- 注意层次总排序也要一致性检验

https://www.bilibili.com/read/cv10175969/ Perron-Frobenius定理-正矩阵最大特征值为正实数

https://zhuanlan.zhihu.com/p/518686257 正互反矩阵最大特征值$\lambda_{max}\geq n$

In [2]:
import numpy as np
A = [[1,1,4,1/3,3],
     [1,1,4,1/3,3],
     [1/4,1/4,1,1/3,1/2],
     [3,3,3,1,3],
     [1/3,1/3,2,1/3,1]]
A = np.asarray(A)
# 检验A是否为正互反矩阵
print(A.T * A == np.ones_like(A))
(np.ones_like(A) != A.T * A).sum() == 0

[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


True

In [3]:
""" 
层次分析法
"""
import numpy as np
class AHP:
  """
  相关信息的传入和准备
  """

  def __init__(self, array):
    ## 记录矩阵相关信息
    self.array = array
    ## 记录矩阵大小
    self.n = array.shape[0]
    # 初始化RI值，用于一致性检验
    self.RI_list = [np.nan, 0, 0, 0.52, 0.89, 1.12, 1.26, 1.36, 1.41, 1.46, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]
    # 矩阵的特征值和特征向量
    self.eig_val, self.eig_vector = np.linalg.eig(self.array)
    # 矩阵的最大特征值
    # self.max_eig_val = np.max(np.abs(self.eig_val)) # 可以用模长算谱半径，但是由于谱半径为正实数，numpy比较的是实部，故直接比较也可以
    self.max_eig_val = np.max(self.eig_val).real
    # 矩阵最大特征值对应的特征向量
    self.max_eig_vector = self.eig_vector[:, np.argmax(self.eig_val)].real
    # 矩阵的一致性指标CI
    self.CI_val = (self.max_eig_val - self.n) / (self.n - 1)
    # 矩阵的一致性比例CR
    self.CR_val = self.CI_val / (self.RI_list[self.n])
    # 检验正互反矩阵
    assert (np.ones_like(A) != A.T * A).sum() == 0

  """
  一致性判断
  """

  def test_consist(self):
    # 打印矩阵的一致性指标CI和一致性比例CR
    print("判断矩阵的CI值为：" + str(self.CI_val))
    # 进行一致性检验判断
    if self.n == 2: # 当只有两个子因素的情况
      print("仅包含两个子因素，不存在一致性问题")
    else:
      if self.CR_val < 0.1: # CR值小于0.1，可以通过一致性检验
        print("判断矩阵的CR值为：" + str(self.CR_val) + "，通过一致性检验")
        return True
      else: # CR值大于0.1, 一致性检验不通过
        print("判断矩阵的CR值为：" + str(self.CR_val) + "，未通过一致性检验")
        return False

  """
  算术平均法求权重
  """

  def cal_weight_by_arithmetic_method(self):
    # 求矩阵的每列的和
    col_sum = np.sum(self.array, axis=0)
    # 将判断矩阵按照列归一化
    array_normed = self.array / col_sum
    # 计算权重向量，对每行取平均值
    array_weight = np.mean(array_normed, axis=1)
    # 打印权重向量
    print("算术平均法计算得到的权重向量w为：\n", array_weight)
    # 返回权重向量的值
    return array_weight

  """
  几何平均法求权重
  """

  def cal_weight__by_geometric_method(self):
    # 求矩阵的每列的积（归一化建议使用比例变换法（不要出现0））
    col_product = np.product(self.array, axis=0)
    # 将得到的积向量的每个分量进行开n次方
    array_power = np.power(col_product, 1 / self.n)
    # 将列向量归一化
    array_weight = array_power / np.sum(array_power)
    # 打印权重向量
    print("几何平均法计算得到的权重向量为：\n", array_weight)
    # 返回权重向量的值
    return array_weight

  """
  特征值法求权重
  """

  def cal_weight__by_eigenvalue_method(self):
    # 将矩阵最大特征值对应的特征向量进行归一化处理就得到了权重
    array_weight = self.max_eig_vector / np.sum(self.max_eig_vector)
    # 打印权重向量
    print("特征值法计算得到的权重向量为：\n", array_weight)
    # 返回权重向量的值
    return array_weight


if __name__ == "__main__":
    # 给出判断矩阵
    A = np.array([[1, 2, 6],
                [1/2, 1, 4],
                [1/6, 1/4, 1]])
    print("初始输入的矩阵为：\n",A)
    test = AHP(A).test_consist()
    # 算术平均法求权重
    weight1 = AHP(A).cal_weight_by_arithmetic_method()
    # 几何平均法求权重
    weight2 = AHP(A).cal_weight__by_geometric_method()
    # 特征值法求权重
    weight3 = AHP(A).cal_weight__by_eigenvalue_method()

初始输入的矩阵为：
 [[1.         2.         6.        ]
 [0.5        1.         4.        ]
 [0.16666667 0.25       1.        ]]
判断矩阵的CI值为：0.00460135635713943
判断矩阵的CR值为：0.008848762225268134，通过一致性检验
算术平均法计算得到的权重向量w为：
 [0.58694639 0.32377622 0.08927739]
几何平均法计算得到的权重向量为：
 [0.10614613 0.19288031 0.70097357]
特征值法计算得到的权重向量为：
 [0.5876311  0.32338586 0.08898305]


In [8]:
A = np.array([[1, 2, 6],
              [1/2, 1, 4],
              [1/6, 1/4, 1]])
np.max(np.linalg.eig(A)[0]).real

3.009202712714279

In [ ]:
np.max([1+1j, 1-2j, 1.1+0j, -2+100j])

(1.1+0j)

## 层次分析法+熵权法

> https://www.zhihu.com/question/274750701/answer/376687323 
> 
> http://www.huaxuejia.cn/ism/CESAISM/topsis_saism.php

- 直接将熵权法计算得到的客观赋权直接作为判断矩阵的**单排序权重**。最后再计算AHP的总排序权重。

## 模糊层次分析法 FAHP

> 有懂模糊层次分析法的亲吗？ - 疯狂绅士的回答 - 知乎 https://www.zhihu.com/question/31677376/answer/2165760263
> 
> http://www.huaxuejia.cn/ism/dematelism/G-sharpen-dematel.php?data=1


- 将多个专家的打分合成一个正互反矩阵